In [1]:
import pandas as pd
import numpy as np
import h5py
import math

from keras.models import Sequential, model_from_json
from keras.layers import Dense, Dropout, Activation, regularizers, Flatten
from keras.callbacks import CSVLogger

from sklearn import ensemble, preprocessing, multiclass
from sklearn.preprocessing import Imputer
from sklearn.model_selection import cross_val_score, train_test_split

from collections import Counter

Using TensorFlow backend.


In [2]:
# Transformation

def sqrt(col):
    return list(map(np.sqrt, col));

def freq(col):
    col = np.floor(col)
    counter = Counter(col)
    return [counter.get(elem) for elem in col]

def tanh(col):
    return list(map(np.tanh, col));

def log(col):
    return list(map(np.log, col));

def my_sigmoid(x):
    return 1 / (1 + math.exp(-x))

def sigmoid(col):
    return list(map(my_sigmoid, col))

def square(col):
    return list(map(np.square, col))

def normalize_val(num, col_min, col_max, norm_range):
    width = col_max-col_min

    if(width == 0): width = 1

    return (num-col_min)/width * (norm_range[1]-norm_range[0]) + norm_range[0]

def normalize(col):
    norm_range = (-1,1)
    col_max = np.amax(col)
    col_min = np.amin(col)
    
    return list(map(lambda x : normalize_val(x, col_min, col_max, norm_range), col))
    


In [110]:
# Globals

# Datasets
#dids = np.load("datasets/indexes.npy")
dids = np.load("magicindexes.npy")

# RF model parameters
seed = 67
#transformations = [sqrt, freq]
#transformations_name = ["sqrt", "freq"]
transformations = [sqrt, freq, tanh, log, sigmoid, square, normalize]
transformations_name = ["sqrt", "freq","tanh", "log", "sigmoid", "square", "normalize"]
trans2target1 = {}
trans2target2 = {}
trans2target3 = {}

# Comrpessed Dataset paramters
qsa_representation = []
num_bin = 10
too_big = 10000

# Neural Nets Parameters and Variables
MLP_LFE_Nets = {}
inp_shape = (2,num_bin)
dropout = 0.2
norm = (0, 10)
pred_threshold = 0.51
train_set_max = 80000
test_set_max = 80000


In [4]:
#def binarize_dataset():

def load_dataset(id):
    X = np.load("datasets/binary_numeric/" + str(id) + "-data.npy")
    y = np.load("datasets/binary_numeric/" + str(id) + "-target.npy")
    categorical = np.load("datasets/binary_numeric/" + str(id) + "-categorical.npy")
    return X,y,categorical

    
def evaluate_model(X, y, categorical):
    imp = Imputer(missing_values="NaN")
    X = imp.fit_transform(X)
    enc = preprocessing.OneHotEncoder(categorical_features=categorical)
    X = enc.fit_transform(X)
    clf = ensemble.RandomForestClassifier(random_state=seed)
    #clf_ovsr = multiclass.OneVsRestClassifier(clf, n_jobs=-1)
    
    return cross_val_score(clf, X, y,cv=10)
    
def is_positive(X,y,categorical,base_score,transformation,feature):
    transformed_feature = np.array(transformation(X[:,feature]))
    X = np.c_[X,transformed_feature]
    categorical = np.append(categorical,False)
    new_score = evaluate_model(X,y,categorical).mean()
    
    return 1 if(base_score <= (new_score - 0.01)) else 0

def is_positive_2(X, y, categorical, base_score, transformation, feature):
    transformed_feature = np.array(transformation(X[:,feature]))
    new_score = evaluate_model(transformed_feature.reshape(-1,1),y,[False]).mean()
    
    return 1 if(base_score <= (new_score - 0.005)) else 0

def is_positive_3(X, y, categorical, base_score, transformation, feature):
    transformed_feature = np.array(transformation(X[:,feature]))
    new_score = evaluate_model(transformed_feature.reshape(-1,1),y,[False]).mean()
    
    return 1 if(new_score > base_score*1.01) else 0
    
    

In [50]:
# Build the target for the compressed feature
bad_datasets = []

def build_target_for_compressed(dids):

    for transf in transformations:
        trans2target1[transf] = []
        trans2target2[transf] = []
        trans2target3[transf] = []

    
    for did in dids:
        print("Start dataset number", did)
        
        try:

            X, y, categorical = load_dataset(did)       

            new_indexes = []

            if(X.shape[0] > too_big):
                new_indexes = np.random.choice(X.shape[0], too_big, replace=False)
                X = X[new_indexes]
                y = y[new_indexes]

            base_score = evaluate_model(X, y, categorical).mean()

            # Find the indexes of numeric attributes
            numerical_indexes = np.where(np.invert(categorical))[0]
            sample_numerical_indexes = np.random.choice(numerical_indexes, min(numerical_indexes.shape[0],10), replace=False)

            for i,transf in enumerate(transformations):
                for feature in sample_numerical_indexes:

                    print("\tEvaluating feature " + str(feature))

                    mlp_target_1 = is_positive(X,y,categorical,base_score,transf, feature)
                    mlp_target_2 = is_positive_2(X,y,categorical,base_score,transf, feature)
                    mlp_target_3 = is_positive_3(X,y,categorical,base_score,transf, feature)

                    print("\t\t" + str(mlp_target_1), str(mlp_target_2), str(mlp_target_3))

                    trans2target1[transf].append((did,feature,mlp_target_1))
                    trans2target2[transf].append((did,feature,mlp_target_2))
                    trans2target3[transf].append((did,feature,mlp_target_3))

        except:
                print("The evaluation of dataset " + str(did) + " failed")
                bad_datasets.append(did)
                continue

In [51]:
# Save the result
def save_target_for_compressed(path):

    for transf, name in zip(transformations, transformations_name):
        np.save(path + name + "1", trans2target1[transf])
        np.save(path + name + "2", trans2target2[transf])
        np.save(path + name + "3", trans2target3[transf])


In [52]:
def normalize_Rx(matrix):
    
    Rxc = np.zeros(shape=matrix.shape)
    
    for i,row in enumerate(matrix):
        max_c = np.amax(row)
        min_c = np.amin(row)
        bin_width = (max_c-min_c)/(norm[1]-norm[0])
        Rxc[i] = np.apply_along_axis(lambda x : np.floor((x-min_c)/(bin_width)+norm[0]), 0, row)
    
    return Rxc

def to_quantile_sketch_array(did, col, targets, bins, t_class, index):
    max_c = np.nanmax(col)
    min_c = np.nanmin(col)
    bin_width = (max_c-min_c)/num_bin
    Rx = np.zeros(shape=(2,num_bin))
    
    if(bin_width == 0):
        return
    
    for val,y in zip(col,targets):
        if not np.isnan(val):
            bin_value = int(np.floor((val-min_c)/bin_width))
            bin_value = np.clip(bin_value, 0, num_bin-1)
            my_class = 0 if t_class == y else 1
            Rx[my_class][bin_value] = Rx[my_class][bin_value] + 1
            
    Rx = normalize_Rx(Rx)

    qsa_representation.append(np.insert(Rx.flatten(), 0, [did,index]))

In [53]:
# Build the compressed dataset
def build_compressed_dataset(dids):

    for did in dids:
        print("Start dataset number", did)

        try:
            X, y, categorical = load_dataset(did)
        
            if(X.shape[0] > too_big):
                new_indexes = np.random.choice(X.shape[0], too_big, replace=False)
                X = X[new_indexes]
                y = y[new_indexes]

            numerical_indexes = np.where(np.invert(categorical))[0]

            classes = set(y)

            for t_class in classes:
                for index in numerical_indexes:
                    to_quantile_sketch_array(did,X[:,index], y, num_bin, t_class, index)
                
        except:
            print("Error with dataset " + str(did))
            continue;
        
    

In [54]:
# Save the compressed datasets
def save_compressed_dataset(path):
    np.save(path + "compressed.npy", qsa_representation)

In [55]:
# CREATING THE NEURAL NETS

def initialize_MLPs():
    
    for transf in transformations_name:
        model = Sequential()

        model.add(Dense(64, input_shape=inp_shape, W_regularizer=regularizers.l2(0.01)))
        model.add(Activation('softmax'))
        model.add(Dropout(dropout))
        model.add(Dense(64))
        model.add(Flatten())
        model.add(Dense(output_dim=1))
        model.add(Activation('relu'))

        # For a binary classification problem
        model.compile(optimizer='rmsprop',
                      loss='binary_crossentropy',
                      metrics=['accuracy'])

        MLP_LFE_Nets[transf] = model
    

In [145]:

def load_compressed_ds():
    data = pd.DataFrame(np.load("datasets/compressed/compressed.npy"))
    data.columns = [str(i) for i in range (num_bin*2+2)]
    return data


def assemble_training_set(compressed, transformation_targets):
    targetDf = pd.DataFrame(transformation_targets)
    targetDf.columns = ["0","1", "2"]
    merged = pd.merge(compressed, targetDf, how='left', on=["0", "1"])
    class_1 = merged.ix[:,2:num_bin + 2].values
    class_2 = merged.ix[:,num_bin + 2:-1].values
    target = np.array(merged.ix[:,-1].values)
    meta_inf = np.array(merged.ix[:,:2].values)
    meta_target = np.c_[target, meta_inf]
    X = []
    
    for c1, c2 in zip(class_1, class_2):
        X.append([c1,c2])    

    return np.array(X), np.array(meta_target)


def split_training_test():
    compressed_ds = load_compressed_ds()

    for transf, name in zip(transformations, transformations_name):
        transformation_targets = np.load("datasets/compressed/" + name + "3.npy")
        X,y = assemble_training_set(compressed_ds, transformation_targets)
        X_s_tr, X_s_test, y_s_tr, y_s_test = train_test_split(X, y, test_size=0.3)
        
        # Dropping the meta-info from training set
        y_s_tr = y_s_tr[:,:1]
        
        np.save("datasets/training/" + name + "-data_split",X_s_tr)
        np.save("datasets/training/" + name + "-target_split",y_s_tr)
        np.save("datasets/test/" + name + "-data_split",X_s_test)
        np.save("datasets/test/" + name + "-target_split",y_s_test)
 
        
def load_training_set(transf):
    
    X = np.load("datasets/training/" + transf + "-data_split.npy")
    y = np.load("datasets/training/" + transf + "-target_split.npy")
    
    return X,y


In [57]:
def balance_dataset(X, y, pos_perc = 0.5):
    X = np.array(X)
    y = np.array(y)

    cnt = Counter(y)

    neg_num = cnt[0]
    pos_num = cnt[1]

    neg_index = (y == 0)
    pos_index = (y == 1)

    X_pos = X[pos_index]
    X_neg = X[neg_index]
    y_pos = y[pos_index]
    y_neg = y[neg_index]

    if pos_perc >= 0.5:
        if pos_num <= neg_num:
            sample_neg_num = int(X_pos.shape[0] * (1/pos_perc - 1))
            neg_sampled_indexes = np.random.choice(X_neg.shape[0], sample_neg_num, replace=False)
            X_neg = X_neg[neg_sampled_indexes]
            y_neg = y_neg[neg_sampled_indexes]

        else:
            sample_neg_ideal_size = int(X_pos.shape[0] * (1/pos_perc - 1))

            if(sample_neg_ideal_size > X_neg.shape[0]):
                sample_pos_num = int(X_neg.shape[0] * (1/(1-pos_perc) - 1))
                pos_sampled_indexes = np.random.choice(X_pos.shape[0], sample_pos_num, replace=False)
                X_pos = X_pos[pos_sampled_indexes]
                y_pos = y_pos[pos_sampled_indexes]
            else:
                neg_sampled_indexes = np.random.choice(X_neg.shape[0], sample_neg_ideal_size, replace=False) 
                X_neg = X_neg[neg_sampled_indexes]
                y_neg = y_neg[neg_sampled_indexes]
    else:
        if pos_num <= neg_num:
            sample_pos_ideal_size = int(X_neg.shape[0] * (1/(1-pos_perc) - 1))

            if(sample_pos_ideal_size > X_pos.shape[0]):
                sample_neg_num = int(X_pos.shape[0] * (1/pos_perc - 1))
                neg_sampled_indexes = np.random.choice(X_neg.shape[0], sample_neg_num, replace=False)
                X_neg = X_neg[neg_sampled_indexes]
                y_neg = y_neg[neg_sampled_indexes]
            else:
                pos_sampled_indexes = np.random.choice(X_pos.shape[0], sample_pos_ideal_size, replace=False) 
                X_pos = X_pos[pos_sampled_indexes]
                y_pos = y_pos[pos_sampled_indexes]
        else:
            sample_pos_num = int(X_neg.shape[0] * (1/(1-pos_perc) - 1))
            pos_sampled_indexes = np.random.choice(X_pos.shape[0], sample_pos_num, replace=False)
            X_pos = X_pos[pos_sampled_indexes]
            y_pos = y_pos[pos_sampled_indexes]

    X = np.concatenate((X_pos, X_neg), axis=0)
    y = np.concatenate((y_pos, y_neg) , axis=0)      

    shuffle_index = np.random.choice(X.shape[0], X.shape[0], replace=False)

    X = X[shuffle_index]
    y = y[shuffle_index]

    return X,y

In [58]:
# Training the nets

def train_MLPs():

    for transf, name in zip(transformations, transformations_name):
        print("Start training " + name)
        
        X, y = load_training_set(name)
        
        #if(X.shape[0] > train_set_max):
        #        new_indexes = np.random.choice(X.shape[0], too_big, replace=False)
        #        X = X[new_indexes]
        #        y = y[new_indexes]
        
        X, y = balance_dataset(X, y.reshape(y.shape[0]), pos_perc=0.5)
        print("dataset_balanced")
        
        y = y.reshape(y.shape[0],1)

        MLP_LFE_Nets[name].summary()
        print ("Inputs: {}".format(MLP_LFE_Nets[name].input_shape))
        print ("Outputs: {}".format(MLP_LFE_Nets[name].output_shape))
        print ("Actual input: {}".format(X.shape))
        print ("Actual output: {}".format(y.shape))
        
        csv_logger = CSVLogger("datasets/MLPs/log/log-"+ name +".csv", append=True, separator=';')
        MLP_LFE_Nets[name].fit(X, y, epochs=500, verbose=1, validation_split=0.2 ,callbacks=[csv_logger]) 


In [59]:

def save_MLPs():
    for transf in transformations_name:
        MLP_LFE_Nets[transf].save_weights("datasets/MLPs/" + transf + "-weights")
        model_json = MLP_LFE_Nets[transf].to_json()
        with open("datasets/MLPs/" + transf + "-net_model", "w") as f:
            f.write(model_json)

def load_MLPs():
    
    for name in transformations_name:
        json_file = open('datasets/MLPs/' + name + "-net_model", 'r')
        loaded_model_json = json_file.read()
        json_file.close()
        loaded_model = model_from_json(loaded_model_json)
        
        # load weights into new model
        loaded_model.load_weights("datasets/MLPs/" + name + "-weights")

        # evaluate loaded model on test data
        loaded_model.compile(optimizer='rmsprop',
                             loss='binary_crossentropy',
                             metrics=['accuracy'])
        
        MLP_LFE_Nets[name] = loaded_model


In [60]:
def load_test_set():
    
    X = np.load("datasets/test/" + transformations_name[0] + "-data_split.npy")
    y_meta = np.load("datasets/test/" + transformations_name[0] + "-target_split.npy")
    t = np.full((y_meta.shape[0],1), 0)
    y_meta = np.concatenate((y_meta, t), axis=1)
    
    for i,name in enumerate(transformations_name[1:]):
        X = np.concatenate((X, np.load("datasets/test/" + name + "-data_split.npy")), axis=0)
        y_meta_tmp = np.load("datasets/test/" + name + "-target_split.npy")
        t = np.full((y_meta_tmp.shape[0],1),i+1)
        y_meta_tmp = np.concatenate((y_meta_tmp, t), axis=1)
        y_meta = np.concatenate((y_meta, y_meta_tmp), axis=0)
    
    return X,y_meta

In [121]:
def evaluate_transformation_classifier():
    
    # Number of prediction on features
    num_of_prediction = {}
    # Number of correct prediction on features
    num_of_correct_prediction = {}
    # Number of dataset which received a prediction
    good_predicted_dids = set()
    num_of_predicted_dataset = 0
    
    pred_mat = []
    
    X, y_meta = load_test_set()
    
    if(X.shape[0] > test_set_max):
        new_indexes = np.random.choice(X.shape[0], too_big, replace=False)
        X = X[new_indexes]
        y_meta = y_meta[new_indexes]
    
    for transf in transformations_name:
        pred_mat.append(MLP_LFE_Nets[transf].predict(X))
        num_of_prediction[transf] = 0
        num_of_correct_prediction[transf] = 0
        
    pred_mat = np.array(pred_mat).transpose()
    
    for predictions,did,feature in zip(pred_mat[0],y_meta[:,1], y_meta[:,2]):
        pmax = np.amax(predictions)
        print(predictions)
        
        if pmax > pred_threshold:
            index = np.where(predictions==pmax)[0][0]
            print(index)
            num_of_prediction[transformations_name[index]] += 1
            
            # Select the target for the transformation and the dataset
            positive_example_found =  np.where((y_meta[:,0] == 1)       &  \
                                               (y_meta[:,1] == did)     &  \
                                               (y_meta[:,2] == feature) &  \
                                               (y_meta[:,3] == index))     \
                                                                       [0].shape[0] > 0
            
            if(positive_example_found):
                print("found!")
                good_predicted_dids.add(did) 
                num_of_correct_prediction[transformations_name[index]] += 1
    
    
    global_correct_pred = 0;
    global_pred = 0
    
    for transf in transformations_name:
        
        if(num_of_prediction[transf] == 0):
            print("No predictions have been made")
            continue
            
        global_pred += num_of_prediction[transf];
        global_correct_pred += num_of_correct_prediction[transf]
    
        print("Evalutation of the transformation classifier: " + transf)
        print("\tNumber of prediction:", num_of_prediction[transf])
        print("\tNumber of Correct prediciton:", num_of_correct_prediction[transf])
        print("Accuracy:", num_of_correct_prediction[transf]/num_of_prediction[transf])
        
    print("\n")    
    print("Number of datasets who received a good prediction:", len(good_predicted_dids))
    print("Total number of positive examples: " + str(np.where(y_meta[:,0] == 1)[0].shape[0]))
    print("Total number of examples: " + str(X.shape[0]))
    print("Total accuracy: " + str(global_correct_pred/global_pred))
    print("Total recall: " + str((global_correct_pred)/(np.where(y_meta[:,0] == 1)[0].shape[0])))

# Build and Preprocess the Dataset

In [ ]:

build_target_for_compressed(dids)
save_target_for_compressed("datasets/compressed/")
    

In [ ]:

build_compressed_dataset(dids)
save_compressed_dataset("datasets/compressed/")


# Train Test Split

In [146]:

split_training_test()


C:\Users\Andrea\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate_ix
  if sys.path[0] == '':



# Train the MLPs


In [147]:

initialize_MLPs()


C:\Users\Andrea\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(64, input_shape=(2, 10), kernel_regularizer=<keras.reg...)`
  
C:\Users\Andrea\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(units=1)`
  del sys.path[0]


In [148]:

train_MLPs()


Start training sqrt
dataset_balanced
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_139 (Dense)            (None, 2, 64)             704       
_________________________________________________________________
activation_93 (Activation)   (None, 2, 64)             0         
_________________________________________________________________
dropout_47 (Dropout)         (None, 2, 64)             0         
_________________________________________________________________
dense_140 (Dense)            (None, 2, 64)             4160      
_________________________________________________________________
flatten_47 (Flatten)         (None, 128)               0         
_________________________________________________________________
dense_141 (Dense)            (None, 1)                 129       
_________________________________________________________________
activation_94 (Activation)   (None, 1) 

54/54 [==============================] - 0s - loss: 0.4370 - acc: 0.8333 - val_loss: 0.9583 - val_acc: 0.6429
Epoch 57/500
54/54 [==============================] - 0s - loss: 0.4599 - acc: 0.8333 - val_loss: 0.7970 - val_acc: 0.6429
Epoch 58/500
54/54 [==============================] - 0s - loss: 0.4648 - acc: 0.8333 - val_loss: 1.8977 - val_acc: 0.6429
Epoch 59/500
54/54 [==============================] - 0s - loss: 0.4270 - acc: 0.7963 - val_loss: 1.8733 - val_acc: 0.6429
Epoch 60/500
54/54 [==============================] - 0s - loss: 0.4196 - acc: 0.8704 - val_loss: 1.8649 - val_acc: 0.6429
Epoch 61/500
54/54 [==============================] - 0s - loss: 0.4040 - acc: 0.8519 - val_loss: 1.8754 - val_acc: 0.6429
Epoch 62/500
54/54 [==============================] - 0s - loss: 0.4575 - acc: 0.8148 - val_loss: 0.9550 - val_acc: 0.6429
Epoch 63/500
54/54 [==============================] - 0s - loss: 0.4356 - acc: 0.8148 - val_loss: 1.7879 - val_acc: 0.6429
Epoch 64/500
54/54 [=========

54/54 [==============================] - 0s - loss: 0.3917 - acc: 0.8333 - val_loss: 2.7568 - val_acc: 0.6429
Epoch 123/500
54/54 [==============================] - 0s - loss: 0.3952 - acc: 0.7593 - val_loss: 3.6472 - val_acc: 0.6429
Epoch 124/500
54/54 [==============================] - ETA: 0s - loss: 0.3939 - acc: 0.750 - 0s - loss: 0.3357 - acc: 0.7778 - val_loss: 3.6420 - val_acc: 0.6429
Epoch 125/500
54/54 [==============================] - 0s - loss: 0.3644 - acc: 0.7593 - val_loss: 3.6441 - val_acc: 0.6429
Epoch 126/500
54/54 [==============================] - 0s - loss: 0.3004 - acc: 0.7593 - val_loss: 3.6410 - val_acc: 0.6429
Epoch 127/500
54/54 [==============================] - 0s - loss: 0.2911 - acc: 0.8148 - val_loss: 3.6371 - val_acc: 0.6429
Epoch 128/500
54/54 [==============================] - 0s - loss: 0.3433 - acc: 0.8333 - val_loss: 3.6362 - val_acc: 0.6429
Epoch 129/500
54/54 [==============================] - 0s - loss: 0.3081 - acc: 0.8148 - val_loss: 3.6284 - 

54/54 [==============================] - 0s - loss: 0.2951 - acc: 0.7778 - val_loss: 3.6273 - val_acc: 0.6429
Epoch 188/500
54/54 [==============================] - 0s - loss: 0.2685 - acc: 0.7593 - val_loss: 3.6268 - val_acc: 0.6429
Epoch 189/500
54/54 [==============================] - 0s - loss: 0.2924 - acc: 0.7778 - val_loss: 3.6249 - val_acc: 0.6429
Epoch 190/500
54/54 [==============================] - 0s - loss: 0.2796 - acc: 0.7778 - val_loss: 3.6212 - val_acc: 0.6429
Epoch 191/500
54/54 [==============================] - 0s - loss: 0.3542 - acc: 0.7222 - val_loss: 3.6219 - val_acc: 0.6429
Epoch 192/500
54/54 [==============================] - 0s - loss: 0.2663 - acc: 0.9259 - val_loss: 3.6195 - val_acc: 0.6429
Epoch 193/500
54/54 [==============================] - 0s - loss: 0.3025 - acc: 0.8519 - val_loss: 3.6138 - val_acc: 0.6429
Epoch 194/500
54/54 [==============================] - 0s - loss: 0.3248 - acc: 0.7963 - val_loss: 3.6109 - val_acc: 0.6429
Epoch 195/500
54/54 [=

54/54 [==============================] - 0s - loss: 0.2520 - acc: 0.8519 - val_loss: 3.5915 - val_acc: 0.6429
Epoch 254/500
54/54 [==============================] - 0s - loss: 0.2773 - acc: 0.7963 - val_loss: 3.5968 - val_acc: 0.7143
Epoch 255/500
54/54 [==============================] - 0s - loss: 0.3146 - acc: 0.8148 - val_loss: 3.5971 - val_acc: 0.6429
Epoch 256/500
54/54 [==============================] - 0s - loss: 0.2924 - acc: 0.7963 - val_loss: 3.5983 - val_acc: 0.6429
Epoch 257/500
54/54 [==============================] - 0s - loss: 0.2525 - acc: 0.8148 - val_loss: 3.5950 - val_acc: 0.5714
Epoch 258/500
54/54 [==============================] - 0s - loss: 0.3629 - acc: 0.7222 - val_loss: 3.5888 - val_acc: 0.6429
Epoch 259/500
54/54 [==============================] - 0s - loss: 0.3023 - acc: 0.6852 - val_loss: 3.5846 - val_acc: 0.7143
Epoch 260/500
54/54 [==============================] - 0s - loss: 0.3277 - acc: 0.7963 - val_loss: 3.5781 - val_acc: 0.6429
Epoch 261/500
54/54 [=

...

35/35 [==============================] - 0s - loss: 0.1523 - acc: 0.8000 - val_loss: 0.5605 - val_acc: 0.5556
Epoch 189/500
35/35 [==============================] - 0s - loss: 0.2055 - acc: 0.9143 - val_loss: 0.5233 - val_acc: 0.5556
Epoch 190/500
35/35 [==============================] - 0s - loss: 0.1949 - acc: 0.9143 - val_loss: 0.5131 - val_acc: 0.5556
Epoch 191/500
35/35 [==============================] - 0s - loss: 0.1710 - acc: 0.8857 - val_loss: 0.6706 - val_acc: 0.5556
Epoch 192/500
35/35 [==============================] - 0s - loss: 0.2445 - acc: 0.8571 - val_loss: 0.5734 - val_acc: 0.5556
Epoch 193/500
35/35 [==============================] - 0s - loss: 0.1327 - acc: 0.8000 - val_loss: 0.7181 - val_acc: 0.5556
Epoch 194/500
35/35 [==============================] - 0s - loss: 0.1986 - acc: 0.7429 - val_loss: 0.5365 - val_acc: 0.5556
Epoch 195/500
35/35 [==============================] - 0s - loss: 0.2181 - acc: 0.7714 - val_loss: 0.5658 - val_acc: 0.5556
Epoch 196/500
35/35 [=

35/35 [==============================] - 0s - loss: 0.1015 - acc: 0.8000 - val_loss: 0.5251 - val_acc: 0.6667
Epoch 254/500
35/35 [==============================] - 0s - loss: 0.1736 - acc: 0.7714 - val_loss: 0.4222 - val_acc: 0.6667
Epoch 255/500
35/35 [==============================] - 0s - loss: 0.1498 - acc: 0.6571 - val_loss: 0.3962 - val_acc: 0.6667
Epoch 256/500
35/35 [==============================] - 0s - loss: 0.1455 - acc: 0.6857 - val_loss: 0.4344 - val_acc: 0.6667
Epoch 257/500
35/35 [==============================] - 0s - loss: 0.5982 - acc: 0.7143 - val_loss: 0.5180 - val_acc: 0.6667
Epoch 258/500
35/35 [==============================] - 0s - loss: 0.1502 - acc: 0.6571 - val_loss: 0.6703 - val_acc: 0.5556
Epoch 259/500
35/35 [==============================] - 0s - loss: 0.1299 - acc: 0.8000 - val_loss: 0.7856 - val_acc: 0.5556
Epoch 260/500
35/35 [==============================] - 0s - loss: 0.2483 - acc: 0.6857 - val_loss: 0.4693 - val_acc: 0.6667
Epoch 261/500
35/35 [=

35/35 [==============================] - 0s - loss: 0.2295 - acc: 0.6000 - val_loss: 0.2691 - val_acc: 0.6667
Epoch 320/500
35/35 [==============================] - 0s - loss: 0.1454 - acc: 0.6571 - val_loss: 0.3708 - val_acc: 0.6667
Epoch 321/500
35/35 [==============================] - 0s - loss: 0.1080 - acc: 0.6571 - val_loss: 0.4200 - val_acc: 0.6667
Epoch 322/500
35/35 [==============================] - 0s - loss: 0.0741 - acc: 0.6857 - val_loss: 0.3301 - val_acc: 0.6667
Epoch 323/500
35/35 [==============================] - 0s - loss: 0.1137 - acc: 0.6571 - val_loss: 0.3136 - val_acc: 0.6667
Epoch 324/500
35/35 [==============================] - 0s - loss: 0.1028 - acc: 0.6286 - val_loss: 0.4243 - val_acc: 0.6667
Epoch 325/500
35/35 [==============================] - 0s - loss: 0.1109 - acc: 0.6857 - val_loss: 0.4413 - val_acc: 0.6667
Epoch 326/500
35/35 [==============================] - 0s - loss: 0.1426 - acc: 0.7429 - val_loss: 0.5636 - val_acc: 0.6667
Epoch 327/500
35/35 [=

35/35 [==============================] - 0s - loss: 0.1057 - acc: 0.6000 - val_loss: 0.3939 - val_acc: 0.6667
Epoch 385/500
35/35 [==============================] - 0s - loss: 0.1719 - acc: 0.6000 - val_loss: 0.3889 - val_acc: 0.6667
Epoch 386/500
35/35 [==============================] - 0s - loss: 0.1150 - acc: 0.6286 - val_loss: 0.4220 - val_acc: 0.6667
Epoch 387/500
35/35 [==============================] - 0s - loss: 0.1058 - acc: 0.6571 - val_loss: 0.5367 - val_acc: 0.6667
Epoch 388/500
35/35 [==============================] - 0s - loss: 0.0895 - acc: 0.8000 - val_loss: 0.4769 - val_acc: 0.6667
Epoch 389/500
35/35 [==============================] - 0s - loss: 0.0997 - acc: 0.6857 - val_loss: 0.3532 - val_acc: 0.6667
Epoch 390/500
35/35 [==============================] - 0s - loss: 0.1351 - acc: 0.7143 - val_loss: 0.6335 - val_acc: 0.6667
Epoch 391/500
35/35 [==============================] - 0s - loss: 0.0899 - acc: 0.7429 - val_loss: 0.4790 - val_acc: 0.6667
Epoch 392/500
35/35 [=

35/35 [==============================] - 0s - loss: 0.0979 - acc: 0.6000 - val_loss: 0.4496 - val_acc: 0.6667
Epoch 451/500
35/35 [==============================] - 0s - loss: 0.0873 - acc: 0.6857 - val_loss: 0.5670 - val_acc: 0.6667
Epoch 452/500
35/35 [==============================] - 0s - loss: 0.1082 - acc: 0.6571 - val_loss: 0.5747 - val_acc: 0.6667
Epoch 453/500
35/35 [==============================] - 0s - loss: 0.0439 - acc: 0.7143 - val_loss: 0.5786 - val_acc: 0.6667
Epoch 454/500
35/35 [==============================] - 0s - loss: 0.0837 - acc: 0.6857 - val_loss: 1.1022 - val_acc: 0.6667
Epoch 455/500
35/35 [==============================] - 0s - loss: 0.1898 - acc: 0.5429 - val_loss: 0.3890 - val_acc: 0.6667
Epoch 456/500
35/35 [==============================] - 0s - loss: 0.0697 - acc: 0.6571 - val_loss: 0.4044 - val_acc: 0.6667
Epoch 457/500
35/35 [==============================] - 0s - loss: 0.1122 - acc: 0.6000 - val_loss: 0.4461 - val_acc: 0.6667
Epoch 458/500
35/35 [=

In [34]:

save_MLPs()


# Load the nets

In [113]:

load_MLPs()



# Test the nets


In [185]:

evaluate_transformation_classifier()


[ 0.53810817  0.71436608  1.58102798  0.38964516  1.55595911  0.90505064]
2
found!
[ 0.          0.13015899  0.          0.          0.37590387  0.04134598]
[ 0.  0.  0.  0.  0.  0.]
[ 0.07975858  2.08141375  2.49256635  0.7267437   0.          1.97168982]
2
[ 0.6293202   0.          0.          0.          0.          1.18164742]
[ 1.39346373  0.41784298  0.          1.9931922   0.4780544   0.        ]
3
[ 0.30873552  2.06320977  2.44679618  0.66245067  0.          2.03172517]
2
found!
[ 0.18340126  0.          0.          0.          0.          0.05122086]
[ 1.24889839  0.99851573  1.13712645  0.          0.          2.31384349]
5
found!
[ 0.58926255  0.85385728  2.09645319  0.98783159  1.264902    1.12757802]
2
found!
[ 1.33895409  0.71875501  1.15215886  1.29188573  0.95298082  0.41735372]
[ 0.02518345  1.28694999  0.72681534  0.          0.          1.98404694]
5
found!
[ 1.47314095  1.42414665  2.14914322  0.          0.          0.91497207]
2
found!
[ 0.58341008  2.02469015  2.

In [184]:
pred_threshold = 1.55
